In [1]:
import pyspark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1625852704433_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# P1.

In [2]:
from pyspark.sql.functions import col, when, size, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
users = spark.read.json('s3://bigdata-desafio/yelp-data/user.json')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Identificando usarios molestosos.

+ Un usuario molestoso es aquél que su promedio de evaluaciones es menor o igual a 2, tiene en promedio menos de 100 reviews y tiene cero fans.

In [4]:
users = users.withColumn('es_molestoso',
                              when((col('average_stars') <= 2)\
                                & (col('review_count') < 100)\
                                & (col('fans') == 0), 1)\
                                .otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Recodificaciones.

In [5]:
users = users.withColumn('friends_count', size(split(col('friends'), ',')))
users = users.withColumn('elite_count', size(split(col('elite'), ',')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
users = users.drop('friends', 'yelping_since', 'name', 'elite', 'user_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
users.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(average_stars=4.03, compliment_cool=1, compliment_cute=0, compliment_funny=1, compliment_hot=2, compliment_list=0, compliment_more=0, compliment_note=1, compliment_photos=0, compliment_plain=1, compliment_profile=0, compliment_writer=2, cool=25, fans=5, funny=17, review_count=95, useful=84, es_molestoso=0, friends_count=99, elite_count=3)]

#### Medicion usuarios molestos.

In [8]:
users.filter(col("es_molestoso") == 1).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

183676

#### Creando vector assembler.

In [9]:
from pyspark.ml.feature import VectorAssembler

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Defino lista features
feats =  ['compliment_cool', 'compliment_cute', 'compliment_funny', 
          'compliment_hot', 'compliment_list', 'compliment_more', 'compliment_note', 
          'compliment_photos', 'compliment_plain', 'compliment_profile', 'compliment_writer', 
          'cool', 'fans', 'funny', 'review_count', 'useful', 'friends_count', 'elite_count']
# Defino vector assembler
va = VectorAssembler(inputCols=feats, outputCol='vector_features')

users_2 = va.transform(users)
users_3 = users_2.select("es_molestoso", "vector_features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### División muestra conjunto entrenamiento y validación.

In [11]:
train_vector, test_vector = users_3.randomSplit([0.7,0.3], seed=37311)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Entrenamiento modelos.

+ LogisticRegression.

In [12]:
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#Train: fit
lr = LogisticRegression(featuresCol = 'vector_features', labelCol = 'es_molestoso')
fitted_model_lr = lr.fit(train_vector)

#Test: transform
lr_prediction = fitted_model_lr.transform(test_vector)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
lr_evaluator = BinaryClassificationEvaluator(labelCol="es_molestoso", rawPredictionCol="rawPrediction")
lr_auc = lr_evaluator.evaluate(lr_prediction)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
lr_auc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.853646926382871

+ GBTClassifier

In [16]:
#Train: fit
gbt = GBTClassifier(featuresCol = 'vector_features', labelCol = 'es_molestoso')
fitted_model_gbt = gbt.fit(train_vector)

#Test: transform
gbt_prediction = fitted_model_gbt.transform(test_vector)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
gbt_evaluator = BinaryClassificationEvaluator(labelCol="es_molestoso", rawPredictionCol="rawPrediction")
gbt_auc = gbt_evaluator.evaluate(gbt_prediction)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
gbt_auc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8643819091596544

+ DecisionTreeClassifier

In [19]:
#Train: fit
dtc = DecisionTreeClassifier(featuresCol = 'vector_features', labelCol = 'es_molestoso')
fitted_model_dtc = dtc.fit(train_vector)

#Test: transform
dtc_prediction = fitted_model_dtc.transform(test_vector)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
dtc_evaluator = BinaryClassificationEvaluator(labelCol="es_molestoso", rawPredictionCol="rawPrediction")
dtc_auc = dtc_evaluator.evaluate(dtc_prediction)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
dtc_auc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.2703594562529107

+ El mejor modelo en base a su AUC score es el Gradient Boosted Trees. Con 0.864.

+ Identifique cuales son los principales atributos asociados a un usuario molestoso y repórtelos

In [107]:
iterable_to_sort_molestoso = list(zip(list(fitted_model_gbt.featureImportances), feats))
sorted(iterable_to_sort_molestoso)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(0.0, 'compliment_cute'), (0.0, 'compliment_funny'), (0.0, 'compliment_hot'), (0.0, 'compliment_list'), (0.0, 'compliment_more'), (0.0, 'compliment_note'), (0.0, 'compliment_photos'), (0.0, 'compliment_plain'), (0.0, 'compliment_profile'), (0.0, 'compliment_writer'), (0.0, 'elite_count'), (0.0005978506956905386, 'compliment_cool'), (0.04058943399297541, 'friends_count'), (0.07582520184276012, 'funny'), (0.09910869607566228, 'fans'), (0.10378065818072539, 'useful'), (0.13305601616650733, 'cool'), (0.5470421430456789, 'review_count')]

+ Los principales atributos asociados a un usuario molestoso son:
    - review_count
    - cool
    - useful

# P2.

In [22]:
business = spark.read.json('s3://bigdata-desafio/yelp-data/business.json')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:

business = business.withColumn('accepts_insurance',
                               when((col('attributes.AcceptsInsurance') == 'True')\
                                     | (col('attributes.AcceptsInsurance') == "\'True\'")\
                                     | (col('attributes.AcceptsInsurance') == "u\'True\'"), 1)\
                               .otherwise(0))

business = business.withColumn('age_restricted',
                              when((col('attributes.AgesAllowed') == 'allages')\
                                    | (col('attributes.AgesAllowed') == "\'allages\'")\
                                    | (col('attributes.AgesAllowed') == "u\'allages\'"), 1)\
                                .otherwise(0))

business = business.withColumn('allows_alcohol',
                              when((col('attributes.Alcohol') == 'beer_and_wine')\
                                    | (col('attributes.Alcohol') == "\'beer_and_wine\'")\
                                    | (col('attributes.Alcohol') == "u\'beer_and_wine\'")\
                                    | (col('attributes.Alcohol') == 'full_bar')\
                                    | (col('attributes.Alcohol') == "\'full_bar\'")\
                                    | (col('attributes.Alcohol') == "u\'full_bar\'"), 1)\
                            .otherwise(0))

business = business.withColumn('accepts_bitcoin',
                              when((col('attributes.BusinessAcceptsBitcoin') == 'True')\
                                    | (col('attributes.BusinessAcceptsBitcoin') == True)\
                                    | (col('attributes.BusinessAcceptsBitcoin') == "\'True\'")\
                                    | (col('attributes.BusinessAcceptsBitcoin') == "u\'True\'"), 1)\
                            .otherwise(0))

business = business.withColumn('food_related',
                              when((col('categories').rlike('Food'))\
                                    | (col('categories').rlike('Restaurants'))\
                                    | (col('categories').rlike('Bars')), 1)\
                            .otherwise(0))

business = business.withColumn('finance_related',
                              when((col('categories').rlike('Banks'))\
                                    | (col('categories').rlike('Insurance'))\
                                    | (col('categories').rlike('Finance')), 1)\
                            .otherwise(0))

business = business.withColumn('health_related',
                              when((col('categories').rlike('Fitness'))\
                                    | (col('categories').rlike('Hospitals'))\
                                    | (col('categories').rlike('Health')), 1)\
                            .otherwise(0))

business = business.withColumn('allows_smoking',
                              when((col('attributes.Smoking') == '\'yes\'')\
                                    |(col('attributes.Smoking') == 'u\'yes\'')\
                                    |(col('attributes.Smoking') == 'yes')\
                                    |(col('attributes.Smoking') == '\'outdoor\'')\
                                    |(col('attributes.Smoking') == 'u\'outdoor\'')\
                                    |(col('attributes.Smoking') == 'outdoor'), 1)\
                            .otherwise(0))

business = business.withColumn('offers_free_wifi',
                              when((col('attributes.WiFi') == '\'free\'')\
                                    | (col('attributes.WiFi') == 'u\'free\'')\
                                    | (col('attributes.WiFi') == 'free'), 1)\
                            .otherwise(0))

business = business.withColumn('considered_expensive',
                              when((col('attributes.RestaurantsPriceRange2') == 3)\
                                    | (col('attributes.RestaurantsPriceRange2') == 4), 1)\
                            .otherwise(0))

business = business.withColumn('kid_friendly',
                              when((col('attributes.GoodForKids') == 'True')\
                                    | (col('attributes.GoodForKids') == True)\
                                    | (col('attributes.GoodForKids') == "\'True\'")\
                                    | (col('attributes.GoodForKids') == "u\'True\'"), 1)\
                            .otherwise(0))

business = business.withColumn('has_tv',
                              when((col('attributes.HasTV') == 'True')\
                                    | (col('attributes.HasTV') == True)\
                                    | (col('attributes.HasTV') == "\'True\'")\
                                    | (col('attributes.HasTV') == "u\'True\'"), 1)\
                            .otherwise(0))

business = business.withColumn('dog_friendly',
                              when((col('attributes.DogsAllowed') == 'True')\
                                    | (col('attributes.DogsAllowed') == True)\
                                    | (col('attributes.DogsAllowed') == "\True'\'")\
                                    | (col('attributes.DogsAllowed') == "u\'True\'"), 1)\
                            .otherwise(0))

business = business.withColumn('considered_as_loud',
                              when((col('attributes.NoiseLevel') == 'loud')\
                                    | (col('attributes.NoiseLevel') == "\'loud\'")\
                                    | (col('attributes.NoiseLevel') == "u\'loud\'")\
                                    | (col('attributes.NoiseLevel') == "very_loud")\
                                    | (col('attributes.NoiseLevel') == "\'very_loud\'")\
                                    | (col('attributes.NoiseLevel') == "u\'very_loud\'"), 1)\
                            .otherwise(0))

business = business.withColumn('offers_happy_hours',
                              when((col('attributes.HappyHour') == 'True')\
                                    | (col('attributes.HappyHour') == True)\
                                    | (col('attributes.HappyHour') == "\'True\'")\
                                    | (col('attributes.HappyHour') == "u\'True\'"), 1)\
                            .otherwise(0))

business = business.withColumn('label',
                              when(col('is_open') == 0, 1).otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
business = business.drop("is_open","address","attributes","business_id","categories","city","hours","latitude","longitude","name", "postal_code","state")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Creando vector assembler.

In [25]:
# Defino lista features
feats_business =  ['review_count', 'stars', 'accepts_insurance', 'age_restricted', 'allows_alcohol', 
                   'accepts_bitcoin', 'food_related', 'finance_related', 'health_related', 'allows_smoking', 
                   'offers_free_wifi', 'considered_expensive', 'kid_friendly', 'has_tv', 'dog_friendly', 
                   'considered_as_loud', 'offers_happy_hours']
# Defino vector assembler
va_business = VectorAssembler(inputCols=feats_business, outputCol='vector_features_business')

business_2 = va_business.transform(business)
business_3 = business_2.select("label", "vector_features_business")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Division muestra conjunto de validación y de entrenamiento.

In [26]:
train_vector_business, test_vector_business = business_3.randomSplit([0.7,0.3], seed=37311)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Entrenamiento de modelos

+ LogisticRegression.

In [27]:
#Train: fit
lr_business = LogisticRegression(featuresCol = 'vector_features_business', labelCol = 'label')
fitted_model_lr_business = lr_business.fit(train_vector_business)

#Test: transform
lr_prediction_business = fitted_model_lr_business.transform(test_vector_business)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
lr_evaluator_business = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
lr_auc_business = lr_evaluator_business.evaluate(lr_prediction_business)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
lr_auc_business

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6918714256418185

+ GBTClassifier

In [30]:
#Train: fit
gbt_business = GBTClassifier(featuresCol = 'vector_features_business', labelCol = 'label')
fitted_model_gbt_business = gbt_business.fit(train_vector_business)

#Test: transform
gbt_prediction_business = fitted_model_gbt_business.transform(test_vector_business)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
gbt_evaluator_business = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
gbt_auc_business = gbt_evaluator_business.evaluate(gbt_prediction_business)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
gbt_auc_business

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7059640993931542

+ DecisionTreeClassifier

In [33]:
#Train: fit
dtc_business = DecisionTreeClassifier(featuresCol = 'vector_features_business', labelCol = 'label')
fitted_model_dtc_business = dtc_business.fit(train_vector_business)

#Test: transform
dtc_prediction_business = fitted_model_dtc_business.transform(test_vector_business)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
dtc_evaluator_business = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction")
dtc_auc_business = dtc_evaluator_business.evaluate(dtc_prediction_business)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
dtc_auc_business

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5599717354948456

+ El mejor modelo en base a su AUC score es el Gradient Boosted Trees. Con 0.705.

+ Identifique cuales son los principales atributos asociados a un usuario molestoso y repórtelos

In [102]:
iterable_to_sort = list(zip(list(fitted_model_gbt_business.featureImportances),feats_business))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
sorted(iterable_to_sort)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(0.0009682790613979285, 'accepts_bitcoin'), (0.004871078396797879, 'age_restricted'), (0.006714508315772911, 'dog_friendly'), (0.0069346332907087626, 'allows_smoking'), (0.014691109574143596, 'offers_happy_hours'), (0.01609106664711758, 'health_related'), (0.01642963197768825, 'considered_as_loud'), (0.028478166908371575, 'finance_related'), (0.03440955874272693, 'considered_expensive'), (0.04360917418836734, 'has_tv'), (0.04645966527099416, 'offers_free_wifi'), (0.07237911857290917, 'accepts_insurance'), (0.07659871018109396, 'kid_friendly'), (0.0900557814818842, 'allows_alcohol'), (0.1675160681876632, 'food_related'), (0.17306787507519375, 'stars'), (0.20072557412716896, 'review_count')]

+ Los principales atributos asociados con el cierre de un servicio son:
    - review_count
    - stars
    - food_related